<a href="https://colab.research.google.com/github/Michal287/DL-notebook/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library instalations

# Importowanie bibliotek

In [ ]:
#os
from imutils import paths
import os
from datetime import datetime
from google.colab import drive

#tools
import numpy as np
import pickle
import random
import imutils

#sklearn
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

#tensorflow tools
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

#tensorboard
%load_ext tensorboard

#tensorflow architecture
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.models import Sequential, load_model

#cv2
import cv2
from google.colab.patches import cv2_imshow

#plots
import matplotlib.pyplot as plt

#agumentation
import albumentations as A

# Remove logs in previous runs

In [ ]:
!rm -rf ./logs/

# Connect to google drive

In [ ]:
drive.mount('/content/drive')

# Checking GPU


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Download images from google drive

In [ ]:
if not os.path.isdir('data'):
  os.mkdir('data')

if not os.path.isdir('output'):
  os.mkdir('output')

!unzip -q /content/drive/MyDrive/raw.zip -d /content/data

# Settings

In [ ]:
image_width, image_height = 244, 244
BATCH_SIZE = 8
EPOCHS = 30

# Agumentation Examples

In [ ]:
transform = A.Compose([
    #A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1),
    #A.HueSaturationValue(val_shift_limit=(-30, 30), p=1),
    #A.Blur(blur_limit=2, p=.2),
    #A.Rotate(limit=15, p=.2),
    A.RandomCrop(210, 210),
    A.Resize(width=image_width, height=image_height,p=1)
])

image = cv2.imread('data/raw_2/2/image_10.jpg')
image = cv2.resize(image, (image_width, image_height))
image_aug = transform(image=image)["image"]
cv2_imshow(image_aug)
print(image_aug.shape)

# Load Data

In [ ]:
transform = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=.8),
    A.HueSaturationValue(p=.5),
    A.Blur(blur_limit=3, p=.3),
    A.Rotate(limit=15, p=.5)
])

def image_darkening(image):
  #matrix = np.ones(image_crop.shape, dtype="float32") * 255
  #return cv2.subtract(image_crop, matrix)
  image[:, :] = 0
  return image

def join_images(images):
  return np.hstack( (np.asarray( image ) for image in images ) )

def split_images(image, image_width, splitter=0.5):
  width_crop = image_width - int(image_width*splitter)
  return image[:,width_crop:], image[:, :width_crop] 


#Get all files in dictionary
image_paths = list(paths.list_images('data/raw_2'))
#Shuffle data
np.random.shuffle(image_paths)

data = []
labels = []

for image_path in image_paths:
    image = cv2.imread(image_path)
    image = cv2.resize(image, (image_width, image_height))
    image_aug = transform(image=image)["image"]
    image_aug = img_to_array(image_aug)
    image_aug2 = transform(image=image)["image"]
    image_aug2 = img_to_array(image_aug)
    image = img_to_array(image)

    width_crop = image_width - int(image_width*0.5)
    image[:,width_crop:] = 0
    image[:,:int(image_width*0.1)] = 0

    width_crop = image_width - int(image_width*0.5)
    image_aug[:,width_crop:] = 0
    image_aug[:,:int(image_width*0.1)] = 0
    
    data.append(image)
    data.append(image_aug)

    label = image_path.split('/')[-2].split('_')
    labels.append(label)
    labels.append(label)

In [ ]:
for i in data[:100]:
  cv2_imshow(i)

# Normalize data and convert to numpy array

In [ ]:
data = np.array(data, dtype='float') / 255.
labels = np.array(labels)

# Checking Data

In [ ]:
print(f'[INFO] {len(image_paths)} obrazów o rozmiarze: {data.nbytes / (1024 * 1000.0):.2f} MB')
print(f'[INFO] Kształt danych: {data.shape}')

[INFO] 336 obrazów o rozmiarze: 1406.54 MB
[INFO] Kształt danych: (1008, 244, 244, 3)


# Checking labels

In [ ]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)
print(f'[INFO] Etykiety: {mlb.classes_}')

[INFO] Etykiety: ['1' '2' '3']


# Save labels to file

In [ ]:
print('[INFO] Eksport etykiet do pliku...')
with open(r'output/mlb.pickle', 'bw') as file:
    pickle.dump(mlb, file)

[INFO] Eksport etykiet do pliku...


# Split data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=10)

print(f'[INFO] Rozmiar danych treningowych: {X_train.shape}')
print(f'[INFO] Rozmiar danych testowych: {X_test.shape}')

[INFO] Rozmiar danych treningowych: (756, 244, 244, 3)
[INFO] Rozmiar danych testowych: (252, 244, 244, 3)


In [ ]:
cv2_imshow(X_train[3]*255)

In [ ]:
def target_repair(lab):
  target = []

  for i in lab:
    target.append(np.argmax(i))

  return target

y_train, y_test = target_repair(y_train), target_repair(y_test)
y_train, y_test = to_categorical(y_train, 3), to_categorical(y_test, 3)

# Create Architecture

In [ ]:
model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(244, 244, 3)),
        MaxPool2D(pool_size=(2, 2)),
        
        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        MaxPool2D(pool_size=(2, 2)),
        
        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        MaxPool2D(pool_size=(2, 2)),
        
        Flatten(),

        Dense(128, activation='relu'),
        Dense(3, activation='softmax')
    ])

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 244, 244, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 122, 122, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 122, 122, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 61, 61, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 61, 61, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 30, 30, 128)     

# Creating best checkpoint

In [ ]:
dt = datetime.now().strftime('%d_%m_%Y_%H_%M')
filepath = os.path.join('output', 'model_' + dt + '.hdf5')
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', save_best_only=True)

# Connect to tensorboard

In [ ]:
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Trening model

In [ ]:
print(f'[INFO] Trenowanie modelu...')

history = model.fit(
    X_train, 
    y_train, 
    batch_size=BATCH_SIZE,
    validation_data=(X_test, y_test),
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint, tensorboard_callback]
)

[INFO] Trenowanie modelu...
Epoch 1/30
67/67 [==============================] - 7s 99ms/step - loss: 0.6854 - accuracy: 0.6996 - val_loss: 0.3717 - val_accuracy: 0.8370
Epoch 2/30
67/67 [==============================] - 5s 77ms/step - loss: 0.2683 - accuracy: 0.9017 - val_loss: 0.4212 - val_accuracy: 0.8296
Epoch 3/30
67/67 [==============================] - 4s 66ms/step - loss: 0.1206 - accuracy: 0.9641 - val_loss: 0.0992 - val_accuracy: 0.9630
Epoch 4/30
67/67 [==============================] - 4s 64ms/step - loss: 0.0479 - accuracy: 0.9887 - val_loss: 0.0567 - val_accuracy: 0.9852
Epoch 5/30
67/67 [==============================] - 4s 65ms/step - loss: 0.0479 - accuracy: 0.9868 - val_loss: 0.0287 - val_accuracy: 0.9926
Epoch 6/30
67/67 [==============================] - 4s 57ms/step - loss: 0.0517 - accuracy: 0.9849 - val_loss: 0.0613 - val_accuracy: 0.9630
Epoch 7/30
67/67 [==============================] - 4s 57ms/step - loss: 0.0057 - accuracy: 0.9981 - val_loss: 0.0673 - val_ac

# Run Tensorboard

In [ ]:
%tensorboard --logdir logs/fit

# Creating charts

In [ ]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.offline as po


def plot_hist(history, filename):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig = make_subplots(rows=2, cols=1, subplot_titles=('Accuracy', 'Loss'))

    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['accuracy'], name='train_accuracy',
                             mode='markers+lines', marker_color='#f29407'), row=1, col=1)
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_accuracy'], name='valid_accuracy',
                             mode='markers+lines', marker_color='#0771f2'), row=1, col=1)
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='train_loss',
                             mode='markers+lines', marker_color='#f29407'), row=2, col=1)
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='valid_loss',
                             mode='markers+lines', marker_color='#0771f2'), row=2, col=1)

    fig.update_xaxes(title_text='Liczba epok', row=1, col=1)
    fig.update_xaxes(title_text='Liczba epok', row=2, col=1)
    fig.update_yaxes(title_text='Accuracy', row=1, col=1)
    fig.update_yaxes(title_text='Loss', row=2, col=1)
    fig.update_layout(width=1400, height=1000, title=f"Metrics")

    po.plot(fig, filename=filename, auto_open=False)



filename = os.path.join('output', 'report_' + dt + '.html')
print(f'[INFO] Eksport wykresu do pliku {filename}...')
plot_hist(history, filename=filename)

[INFO] Eksport wykresu do pliku output/report_24_10_2022_07_32.html...


# Load test

In [ ]:
if not os.path.isdir('test_data'):
  os.mkdir('test_data')

!unzip -q /content/drive/MyDrive/test_data.zip -d /content/test_data

# Load model and labels

In [ ]:
print('[INFO] Loading model...')
model = load_model('output/model_'+dt+'.hdf5')

print('[INFO] Loading labels...')
with open(r'output/mlb.pickle', 'br') as file:
    mlb = pickle.load(file)

[INFO] Loading model...
[INFO] Loading labels...


In [ ]:
#filename = 'test_data/test_3.jpg'
filename = 'data/raw_2/3/image_143.jpg'

def load(filename):
    image = cv2.imread(filename)
    image = cv2.resize(image, (244, 244))
    image = image.astype('float') / 255.
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    width_crop = image_width - int(image_width*0.6)
    image[:,width_crop:] = 0

    return image

image = load(filename)

y_pred = model.predict(image)[0]

labels = dict(enumerate(mlb.classes_))
print(labels)
idxs = np.argsort(y_pred)

print('[INFO] Loading image...')
image = cv2.imread(filename)

print('[INFO] Displaying image...')
for i, idx in enumerate(idxs):
    cv2.putText(img=image, text=f'Labels: {labels[idx]:6} Probability: {y_pred[idx] * 100:.4f}%',
                org=(10, (i * 30) + 25), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5,
                color=(0, 0, 255), thickness=1)

cv2_imshow(image)

Checking important feautres

In [ ]:
_img = cv2.imread('test_data/test_1.jpg')
_img = cv2.resize(_img, (244, 244))
cv2_imshow(_img)

width_crop = image_width - int(image_width*0.6)
_img[:,width_crop:] = 0
cv2_imshow(_img)

d = _img[44: ,44:]
cv2_imshow(d)

In [ ]:
img = _img.astype('float') / 255.
img = np.expand_dims(img, axis=0)

images = tf.Variable(img, dtype=float)

with tf.GradientTape() as tape:
    pred = model(images, training=False)
    class_idxs_sorted = np.argsort(pred.numpy().flatten())[::-1]
    loss = pred[0][class_idxs_sorted[0]]
    
grads = tape.gradient(loss, images)

In [ ]:
dgrad_abs = tf.math.abs(grads)

In [ ]:
dgrad_max_ = np.max(dgrad_abs, axis=3)[0]

In [ ]:
arr_min, arr_max  = np.min(dgrad_max_), np.max(dgrad_max_)
grad_eval = (dgrad_max_ - arr_min) / (arr_max - arr_min + 1e-18)

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(14,5))
axes[0].imshow(_img)
i = axes[1].imshow(grad_eval,cmap="jet",alpha=0.8)
fig.colorbar(i)
print(np.argmax(pred))

# Removing light

In [ ]:
image_native = cv2.imread('data/raw_2/3/image_1.jpg')

image = cv2.cvtColor(image_native, cv2.COLOR_BGR2HSV)

increase = 1

v = image[:, :, 2]

v = np.where(v < 170, v + increase, 170)

image[:, :, 2] = v

image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

cv2_imshow(image_native)
cv2_imshow(image)